# Step 1 - Frame Dividing Part

#### Install Opencv for Frame Extraction

In [ ]:
pip install opencv-python-headless

#### Importing Libraries CV2 and OS

In [ ]:
import cv2
import os

#### Function to Extract Frames

In [ ]:
def extract_frames(video_path, output_folder, frame_rate=30):
    """Extract frames from a video at a given frame rate."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get video FPS

    # Debug: Print FPS
    print(f"Video: {video_path}, FPS: {fps}")

    if fps == 0:
        print("Error: Could not read FPS. Check the video file or codec.")
        return  # Exit function to prevent division by zero

    frame_interval = max(1, int(fps / frame_rate))  # Avoid division by zero

    count = 0
    frame_id = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop when the video ends

        if count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{frame_id:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            frame_id += 1  # Increment frame number

        count += 1

    cap.release()
    print(f" Extracted {frame_id} frames from {video_path}")


##### Extracting frames

In [ ]:
extract_frames("Video/Player02_FS_Set01_F_50Kg.MOV", "Output Video/frames/front", frame_rate=30)
extract_frames("Video/Player02_FS_Set01_S_50Kg.MOV", "Output Video/frames/side", frame_rate=30)
extract_frames("Video/Player02_FS_Set01_A_50Kg.mp4", "Output Video/frames/top", frame_rate=30)


#### Check the orientation and Quality

In [ ]:
import cv2
import os

def check_orientation(folder_path):
    """Check the orientation of images in a folder."""
    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if image is not loaded

        height, width, _ = img.shape
        orientation = "Landscape" if width > height else "Portrait"
        print(f"{img_name}: {orientation} ({width}x{height})")

# Example Usage
check_orientation("Output Video/frames/front")
check_orientation("Output Video/frames/side")
check_orientation("Output Video/frames/top")


#### Rotate fremes if needed

In [ ]:
def rotate_images(folder_path, rotate_code):
    """Rotate all images in a folder."""
    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue  # Skip corrupted files

        rotated = cv2.rotate(img, rotate_code)
        cv2.imwrite(img_path, rotated)
        print(f"Rotated {img_name}")

# Rotate 90 degrees clockwise
rotate_images("Output Video/frames/front", cv2.ROTATE_90_CLOCKWISE)

# Rotate 90 degrees counter-clockwise
# rotate_images("Output Video/frames/side", cv2.ROTATE_90_COUNTERCLOCKWISE)


#### Remove Blurry Images from frames

In [ ]:
def analyze_blurriness(folder_path):
    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        print(f"{img_name}: Variance = {laplacian_var:.2f}")


In [ ]:
analyze_blurriness("Output Video/frames/top")

In [ ]:
import shutil

def move_blurry_images(folder_path, threshold=100.0, blurry_folder="blurry_top"):
    os.makedirs(blurry_folder, exist_ok=True)
    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()

        if laplacian_var < threshold:
            shutil.move(img_path, os.path.join(blurry_folder, img_name))
            print(f"Moved blurry image: {img_name} (Variance = {laplacian_var:.2f})")
